## Setup

Add any imports needed here

Mount Google Drive

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install datasets
!pip -q install transformers
!pip -q install peft
!pip -q install -U bitsandbytes
!pip -q install rouge


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.8 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper


In [ ]:
import numpy as np
from datasets import load_dataset
import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
from bs4 import BeautifulSoup

drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS6220 Folder #add direct access to the folder if you get an error in this cell
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

import helpers
import importlib

MessageError: ignored

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, folder_rawdata, folder_metadata, problem_nums, delimiter="#"): #initializes the dataset with the folder path, delimiter, and chunk size
        # problem_nums_strings = [str(num).zfill(5) for num in problem_nums]
        self.data = []
        count = 0

        for num_int in problem_nums:
            num_str = str(num_int).zfill(5)
            sample_data_path = os.path.join(folder_rawdata, "p" + num_str + ".py")
            print(sample_data_path)
            sample_meta_path = os.path.join(folder_metadata, "p" + num_str + ".html")

            metadata_string = None
            try:
                with open(sample_meta_path) as f:
                    metadata_string = f.read()

                # meta_string = problem_metadata[num_int]

                metadata_string = metadata_string[:metadata_string.find("<H2>")]
                soup = BeautifulSoup(metadata_string, 'html.parser')
                metadata_string = "\"\"\"\n" + soup.get_text() + "\n" + "\"\"\"\n"

                with open(sample_data_path, 'r', encoding='utf-8') as file:
                    code_samples = file.read().split(delimiter)
                    modified_samples = [metadata_string + str(s) for s in code_samples if len(s) > 10]
                    self.data.extend(modified_samples)
            except:
                continue

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        return self.data[idx]

    # def contains_non_english_characters(file_path):
    #     with open(file_path, 'r') as file:
    #         data = file.read()
    #         for char in data:
    #             print(char)
    #             if not char.isascii() and not re.match(r'[a-zA-Z]', char):
    #                 print(char, file_path)
    #                 return True
    #     return False

In [ ]:
!ls

 6_project_proposal.pdf
'Aggregate and Save.ipynb'
'[Archive] Ronit - CS 6220 Federated Learning on Codebases'
'[Archive] Ronit - Working with IBM Data'
'CodeGen Experiments'
 codeparrot-clean
'CodeParrot Experiments.ipynb'
'CS 6220 Demo Team #6 - Federated Learning for Codebase Q&A.gslides'
'CS 6220 Proposal - Federated Learning for Codebase Q&A.gslides'
'CS 6220 Workshop Team #6 - Federated Learning for Codebase Q&A.gslides'
 helpers
 human_eval
 __init__.py
'Meeting Minutes CS6220.gdoc'
 problem_descriptions
'Ronit - Data and Visualizations.ipynb'
 ronit_ibm_outputs
 ronit_setup
 Scripts_Project_CodeNet_Python800
'Sumanth - MDPP Eval Dataset.ipynb'
 sumanth_setup


In [ ]:
BATCH_SIZE = 16

# Obtain data loaders for each client (separate partitions of data)
print('Loading Data')
client_dataloaders = []
data_path = "./Scripts_Project_CodeNet_Python800"
meta_path = "./problem_descriptions"
delimiter = "#####################################################"

problem_nums_list = [[2388], [2419]]
client_dataloaders = []
for r in problem_nums_list:
    dataset_temp = CustomDataset(data_path, meta_path, r)

    data_loader_temp = torch.utils.data.DataLoader(
        dataset_temp, batch_size=BATCH_SIZE, shuffle=True, drop_last=True
    )

    client_dataloaders.append(data_loader_temp)

print('Finished loading data')

Loading Data
./Scripts_Project_CodeNet_Python800/p02388.py
./Scripts_Project_CodeNet_Python800/p02419.py
Finished loading data


In [ ]:
from helpers.train import generic_training_runner
from transformers import BitsAndBytesConfig
from peft import LoraConfig

model_name = "Salesforce/codegen-2B-mono"
run_title = "salesforcecodegen_manyepochs"

SAVE_PATH = f"./ronit_ibm_outputs/checkpoints/{run_title}"
LOSSES_PATH = f"./ronit_ibm_outputs/logs/losses/{run_title}"
TIMES_PATH = f"./ronit_ibm_outputs/logs/times/{run_title}_elapsed_time"


tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model_info = {
    'model_name': model_name,
    'tokenizer': tokenizer,
    'client_dataloaders': client_dataloaders,
    'quant_config':
        BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        ),
    'lora_config':
        LoraConfig(
            r=8,
            target_modules=["qkv_proj", "out_proj", "fc_in", "fc_out"],
            lora_alpha=32,
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM",
        ),
}

training_args = {
    'clients': 2,
    'data_per_client': 1e3,
    'MAX_LENGTH': 1024,
    'conduct_logging': True,
    'EPOCHS': 5,
    'lr': 4e-4
}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generic_training_runner(
    SAVE_PATH,
    LOSSES_PATH,
    TIMES_PATH,
    model_info,
    training_args,
)

Device Type: cuda
Beginning training iteration
Loaded global model.
Number of batches: 19
Starting training


1it [00:01,  1.63s/it]

Step: 0, Loss: 3.2737


2it [00:04,  2.30s/it]

Step: 1, Loss: 2.9772


...

23it [01:56,  5.09s/it]

Step: 22, Loss: 0.1954


24it [02:01,  5.06s/it]

Step: 23, Loss: 0.1748
Epoch 4 Complete
Ending training
595.2259159088135


MODEL SAVED


## Pretrained model

This model [name] from Hugging Face is pretrained on code.

In [ ]:
importlib.reload(helpers)

from helpers.fl_impl import get_fed_avg_model

SAVE_PATH = f"./ronit_ibm_outputs/checkpoints/{run_title}"

global_aggregated_model = get_fed_avg_model(SAVE_PATH, model_info, num_clients=2)
torch.cuda.empty_cache()

Device Type: cuda
--------------------------------
Processing client 0
Loading model from ./ronit_ibm_outputs/checkpoints/salesforcecodegen_manyepochs_0
Loaded model
Updating state dict
--------------------------------
Processing client 1
Loading model from ./ronit_ibm_outputs/checkpoints/salesforcecodegen_manyepochs_1
Loaded model
Updating state dict


## Evaluation

In [ ]:
import importlib

import helpers
import helpers.evaluation
importlib.reload(helpers)
importlib.reload(helpers.evaluation)

from helpers.evaluation import (
    generate_batch_completions,
    get_humaneval_dataloader,
    generate_model_on_problems,
    get_prompt_from_descr,
    clean_samples,
    compute_average_rouge_scores,
    print_gens_on_problem_all_models,
)

In [ ]:
text = "def is_palindrome(n):"

out = generate_batch_completions([text], global_aggregated_model, tokenizer)
print(out[0])

Device Type: cuda
def is_palindrome(n):
    return str(n) == str(n)[::-1]





In [ ]:
sample_desc = get_prompt_from_descr("/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p00003.html")
print(generate_batch_completions([sample_desc], global_aggregated_model, tokenizer)[0])

Device Type: cuda
"""

Is it a Right Triangle?

Write a program which judges wheather given length of three side form a right triangle. Print "YES" if the given sides (integers) form a right triangle, "NO" if not so.


"""

#!/bin/python3

import math
import os
import random
import re
import sys

# Complete the isRightTriangle function below.
def isRightTriangle(a, b, c):
    if a**2 + b**2 == c**2:
        return "YES"
    else:
        return "NO"




In [ ]:
import human_eval
importlib.reload(human_eval)
from human_eval.data import write_jsonl, read_problems

problems = read_problems()
probs_dataloader = get_humaneval_dataloader(problems, batch_size=10)

In [ ]:
print(problems['HumanEval/7']['prompt'])
print(problems['HumanEval/7']['test'])

from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """



METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([], 'john') == []
    assert candidate(['xxx', 'asd', 'xxy', 'john doe', 'xxxAAA', 'xxx'], 'xxx') == ['xxx', 'xxxAAA', 'xxx']
    assert candidate(['xxx', 'asd', 'aaaxxy', 'john doe', 'xxxAAA', 'xxx'], 'xx') == ['xxx', 'aaaxxy', 'xxxAAA', 'xxx']
    assert candidate(['grunt', 'trumpet', 'prune', 'gruesome'], 'run') == ['grunt', 'prune']



In [ ]:
samples = generate_model_on_problems(global_aggregated_model, probs_dataloader, tokenizer)

write_jsonl("human_eval/samples-sf.json", samples)

Device Type: cuda


KeyboardInterrupt: ignored

In [ ]:
!python3 human_eval/evaluate_functional_correctness.py human_eval/samples.json

Reading samples...
164it [00:00, 3737.28it/s]
Running test suites...
100% 164/164 [00:03<00:00, 47.97it/s]
Writing results to human_eval/samples.json_results.jsonl...
100% 164/164 [00:00<00:00, 13486.77it/s]
{'pass@1': 0.23170731707317074}


In [ ]:
references = [problems[f'HumanEval/{i}']['prompt'] for i in range(len(samples))]
hypotheses = [samples[i]['completion'] for i in range(len(samples))]

# Compute average ROUGE scores
average_rouge_scores = compute_average_rouge_scores(references, hypotheses)
print(average_rouge_scores)

{'rouge-1': {'f': 0.18300701818514034, 'p': 0.18221335345601414, 'r': 0.21576938373381374}, 'rouge-2': {'f': 0.020117647587168327, 'p': 0.019434990438125556, 'r': 0.023989725149679147}, 'rouge-l': {'f': 0.16764480265233261, 'p': 0.1671456622464535, 'r': 0.19763420939721169}}


In [ ]:
print_gens_on_problem_all_models(
    SAVE_PATH,
    "/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p02388.html",
    model_info,
    global_aggregated_model
)

Device Type: cuda
Device Type: cuda
"""

X Cubic

Write a program which calculates the cube of a given integer x.


"""

# x = int(input("Enter a number: "))
#
# print(x**3)

#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

------------------------
Loaded model
Device Type: cuda
"""

X Cubic

Write a program which calculates the cube of a given integer x.


"""

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x
------------------------
Loaded model
Device Type: cuda
"""

X Cubic

Write a program which calculates the cube of a given integer x.


"""

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x = int(input("Enter a number: "))
#
# print(x**3)

# x 

In [ ]:
print_gens_on_problem_all_models(
    SAVE_PATH,
    "/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p02419.html",
    model_info,
    global_aggregated_model
)

Device Type: cuda
Device Type: cuda
"""
Finding a Word

Write a program which reads a word W and a text T, and prints the number of word W which appears in text T


T consists of string Ti separated by space characters and newlines. Count the number of Ti which equals to W. The word and text are case insensitive.


"""

# Solution

#!/bin/python3

import math
import os
import random
import re
import sys

# Complete the findWord function below.
def findWord(s, d, w):
    count = 0
    for i in range(len(s)):
        if s[i] == w:
            count += 1
    if count == d:
        return "YES"
    else:
        return "NO"


------------------------
Loaded model
Device Type: cuda
"""
Finding a Word

Write a program which reads a word W and a text T, and prints the number of word W which appears in text T


T consists of string Ti separated by space characters and newlines. Count the number of Ti which equals to W. The word and text are case insensitive.


"""

# Solution

#!/bin/python3



In [ ]:
sample_desc = get_prompt_from_descr("/content/drive/MyDrive/CS6220 Folder/problem_descriptions/p02419.html")
print(sample_desc)
print(generate_batch_completions([sample_desc], global_aggregated_model, tokenizer)[0])

"""
Finding a Word

Write a program which reads a word W and a text T, and prints the number of word W which appears in text T


T consists of string Ti separated by space characters and newlines. Count the number of Ti which equals to W. The word and text are case insensitive.


"""

Device Type: cuda
"""
Finding a Word

Write a program which reads a word W and a text T, and prints the number of word W which appears in text T


T consists of string Ti separated by space characters and newlines. Count the number of Ti which equals to W. The word and text are case insensitive.


"""

# Solution

#!/bin/python3

import math
import os
import random
import re
import sys

# Complete the countApplesAndOranges function below.
def countApplesAndOranges(s, t, a, b, apples, oranges):
    count_apples = 0
    count_oranges = 0
    for i in range(len(apples)):
        if apples[i] + a >= s and apples[i] + a <= t:
            count_apples += 1
    for i in range(len
